In [ ]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [ ]:
# --- Cell 2 ---
# Initializations
N = 6 # dimension of state
M = 2 # dimension of action

t = 0.             # t0
tf = 30.           # total time
dt = 0.05          # simulation time step
S = int(tf/dt) + 1 # total steps
print(f"S = {S}")

# Parameters
g = 9.81   # gravity (m/s^2)
m = 1.     # mass (m)
J = 0.05   # inertia (kg*m^2)
kx = 0.3
ky = 0.3
k_theta = 0.02

# Limits of thrust (T) and torque (τ)
Tmin = 1/2 * m * g
Tmax = 3/2 * m * g
tau_min = -1/2
tau_max = 1/2

In [ ]:
# --- Cell 3 ---
# Cost
Q = np.diag([1., 1., 10., 0.1, 0.1, 1])
R = 0.1 * np.eye(M)
QN = np.eye(N)

In [ ]:
# --- Cell 4 ---
def simulation(xs, x0, x_ref):
    states = np.hstack(xs)

    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-20, 20)
    ax.set_ylim(-20, 20)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.set_title('Controlling Quadrotor via Finite LQR, with external forces')
    
    L = 1.5
    quadrotor_L,  = ax.plot([], [], 'o-', lw=1, color='black')
    init_point,   = ax.plot([], [], 'bo', markersize=5)
    target_point, = ax.plot([], [], 'ro', markersize=5)
    
    def init():
        quadrotor_L.set_data([], [])
        init_point.set_data([x0[0,0]], [x0[1,0]])
        target_point.set_data([x_ref[0,0]], [x_ref[1,0]])
        
        return quadrotor_L, init_point, target_point
    
    def animate(i):
        # Get current state
        x = states[0, i]
        y = states[1, i]
        theta = states[2, i]
        
        # Left side
        xl = x - L * np.cos(theta)
        yl = y - L * np.sin(theta)
        # Right side
        xr = x + L * np.cos(theta)
        yr = y + L * np.sin(theta)
        
        # Update quadrotor's position
        quadrotor_L.set_data([xl, xr], [yl, yr])
        
        return quadrotor_L,
    
    # Create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=S, interval=20, blit=True)
    # anim.save("LQR_with_Fext.mp4", writer='ffmpeg', fps=30, dpi=100)
    
    plt.close()
    
    return HTML(anim.to_jshtml())

In [ ]:
# --- Cell 5 ---
def dynamics(x, u, Fext=None):
    if Fext is None:
        Fx, Fy, Ftheta = 0., 0., 0.
    else:
        Fx, Fy, Ftheta = Fext[0,0], Fext[1,0], Fext[2,0]
        
    x, y, theta, x_dot, y_dot, theta_dot = x[0,0], x[1,0], x[2,0], x[3,0], x[4,0], x[5,0]
    T, tau = u[0,0], u[1,0]
    
    vx, vy = x_dot, y_dot
    
    fx = -kx * vx * np.abs(vx)
    fy = -ky * vy * np.abs(vy)
    f_theta = -k_theta * theta_dot
    
    x_ddot = 1/m * (-np.sin(theta) * (T + fy) + np.cos(theta) * fx + Fx)
    y_ddot = 1/m * ( np.cos(theta) * (T + fy) + np.sin(theta) * fx + Fy) - g
    theta_ddot = (tau + f_theta + Ftheta) / J
    
    f = np.array([[x_dot, y_dot, theta_dot, x_ddot, y_ddot, theta_ddot]]).T

    return f

def deriv_dynamics(x, u, Fext=None):
    theta = x[2,0]
    T, tau = u[0,0], u[1,0]

    f = dynamics(x, u, Fext)
    x_dot, y_dot, theta_dot = f[0,0], f[1,0], f[2,0]
    
    vx, vy = x_dot, y_dot

    fx = -kx * vx * np.abs(vx)
    fy = -ky * vy * np.abs(vy)
    f_theta = -k_theta * theta_dot
    
    dfx = -2 * kx * np.abs(vx)
    dfy = -2 * ky * np.abs(vy)
    df_theta = k_theta / J

    a = -1/m * ( np.cos(theta) * (T + fy) + np.sin(theta) * fx)
    b =  1/m * (-np.sin(theta) * (T + fy) + np.cos(theta) * fx)
    Ac = np.array([[0., 0., 0.,          1.          ,           0.          ,    0.    ],
                   [0., 0., 0.,          0.          ,           1.          ,    0.    ],
                   [0., 0., 0.,          0.          ,           0.          ,    1.    ],
                   [0., 0., a , np.cos(theta)/m * dfx, -np.sin(theta)/m * dfy,    0.    ],
                   [0., 0., b , np.sin(theta)/m * dfx,  np.cos(theta)/m * dfy,    0.    ],
                   [0., 0., 0.,          0.          ,           0.          , df_theta]])
    
    Bc = np.array([[          0.        ,   0.],
                   [          0.        ,   0.],
                   [          0.        ,   0.],
                   [-1/m * np.sin(theta),   0.],
                   [ 1/m * np.cos(theta),   0.],
                   [          0.        , 1./J]])
    
    return Ac, Bc

In [ ]:
# --- Cell 6 ---
# Runge-Kutta 4th Order integration
def rk4(x, u, dt, Fext=None):
    f1 = dynamics(x, u, Fext)
    f2 = dynamics(x + f1 * dt/2, u, Fext)
    f3 = dynamics(x + f2 * dt/2, u, Fext)
    f4 = dynamics(x + f3 * dt, u, Fext)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

def rk4_deriv(x, u, dt):
    x2 = x + 0.5 * dt * dynamics(x, u)
    x3 = x + 0.5 * dt * dynamics(x2, u)
    x4 = x + dt * dynamics(x3, u)

    dfdx, dfdu = deriv_dynamics(x, u)
    dfdx2, dfdu2 = deriv_dynamics(x2, u)
    dfdx3, dfdu3 = deriv_dynamics(x3, u)
    dfdx4, dfdu4 = deriv_dynamics(x4, u)

    dx2dx = np.eye(N) + 0.5 * dt * dfdx
    dx3dx = np.eye(N) + 0.5 * dt * dfdx2 @ dx2dx
    dx4dx = np.eye(N) + dt * dfdx3 @ dx3dx

    dx2du = 0.5 * dt * dfdu
    dx3du = 0.5 * dt * (dfdu2 + dfdx2 @ dx2du)
    dx4du = dt * (dfdu3 + dfdx3 @ dx3du)
    
    Ad = np.eye(N) + dt/6. * (dfdx + 2. * dfdx2 @ dx2dx + 2. * dfdx3 @ dx3dx + dfdx4 @ dx4dx)
    Bd = dt/6. * (dfdu + 2 * (dfdu2 + dfdx2 @ dx2du) + 2 * (dfdu3 + dfdx3 @ dx3du) + (dfdu4 + dfdx4 @ dx4du))
    
    return Ad, Bd

# **2. Linearization and LQR**

## 2.1 Linearization around a fixed point

**What does it mean we "linearize around a fixed point"?**

Discrete dynamics: $x_{k+1} = f_{discrete}(x_k, u_k)$  
Fixed point: $\bar{x} = f_{discrete}(\bar{x}, \bar{u})$

Taylor Series around $\bar{x}, \bar{u}$:
$$x_{k+1} = f_{discrete}(\bar{x}, \bar{u}) + \underbrace{\left. \frac{\partial f_{discrete}}{\partial x} \right|_{\bar{x}, \bar{u}}}_{A} (x_k - \bar{x}) + \underbrace{\left. \frac{\partial f_{discrete}}{\partial u} \right|_{\bar{x}, \bar{u}}}_{B} (u_k - \bar{u})$$

We can set $x_k = \bar{x} + \Delta x_k$, $u_k = \bar{u} + \Delta u_k$ and thus:
$$\bar{x} + \Delta x_{k+1} = f_{discrete}(\bar{x}, \bar{u}) + A \Delta x_k + B \Delta u_k$$
$$\Delta x_{k+1} = A \Delta x_k + B \Delta u_k$$

## 2.2 Task
In this part, you are asked to linearize the system around the hover state $\bar{x}$ and hover control
input $\bar{u}$ (you need to find the point), and perform LQR on the linearized version of the system. In
particular:

1. Write a function in Python called `linearize()` that takes as input the state $\bar{x}$ and controls
$\bar{u}$ and returns the matrices **A** and **B** for the linearized system (you should use the function
`dynamics()` you created in the first part).

In [ ]:
# --- Cell 7 ---
def linearize(x_bar, u_bar, dt):
    return rk4_deriv(x_bar, u_bar, dt)

2. Write a function in Python called `lqr()` that takes as input the linearized dynamics matrices **A** and **B** along with matrices **$Q_N$**, **Q** and **R**, solves the infinite LQR problem and returns the **P** and **K** matrices. Define your own **$Q_N$**, **Q** and **R** matrices for the planar quadrotor task: here we want to stabilize the system at the hover point.

In [ ]:
# --- Cell 8 ---
# Infinite horizon LQR
def lqr(A, B, QN, Q, R):
    Pk = QN.copy()
    while True:
        lhs = R + B.T @ Pk @ B
        rhs = B.T @ Pk @ A
        K = np.linalg.solve(lhs, rhs)
        P = Q + A.T @ Pk @ (A - B @ K)
        
        if np.allclose(Pk, P, atol=1e-8):
            break
        Pk = P
    
    return K, P

3. Write a simulation loop that controls the system using the infinite LQR controller. **Note that we need to simulate with the actual non-linear dynamics!**  
The simulation timestep should be dt = 0.05. The control inputs need to be limited as described in the previous section. The initial state should be **$\bar{x}$**.

In [ ]:
# --- Cell 9 ---
# Εquilibrium state and control
x_bar = np.array([[0., 0., 0., 0., 0., 0.]]).T
u_bar = np.array([[m*g, 0.]]).T

# Initial state and control
x0 = x_bar
u0 = u_bar

# Reference state and control (the most remote points)
# Without external force and max_vel = 85
# x_ref = np.array([[16., 84.5, 0., 0., 0., 0.]]).T
# x_ref = np.array([[-16., 84.5, 0., 0., 0., 0.]]).T

# Without external force and max_vel = 20
# x_ref = np.array([[15.5, 19.5, 0., 0., 0., 0.]]).T
# x_ref = np.array([[-15.5, 19.5, 0., 0., 0., 0.]]).T

# With external force
x_ref = np.array([[15., 4.5, 0., 0., 0., 0.]]).T

u_ref = np.array([[m*g, 0.]]).T

# External force / Disturbance 
# Fext1 = None
# Fext2 = None
Fext1 = np.array([[-25., -10., 0.02]]).T # [fx, fy, f_theta]
Fext2 = np.array([[9., 10., 0.05]]).T # [fx, fy, f_theta]

# Linearization around (x_bar, u_bar) 
A, B = linearize(x_bar, u_bar, dt)

# LQR
K, _ = lqr(A, B, QN, Q, R)

x = np.copy(x0)
xs = [np.copy(x)]
us = []

for k in range(S-1):
    # Control update
    u = u_bar - K @ (x - x_ref)

    # Clamp u
    T = max(Tmin, min(u[0,0], Tmax))
    tau = max(tau_min, min(u[1,0], tau_max))
    u = np.array([[T, tau]]).T
    us.append(np.copy(u))
    
    # State update (with the actual non-linear dynamics) + Apply external force/disturbance
    if ((60 <= k <= 80) and (Fext1 is not None)):
        x = rk4(x, u, dt, Fext1)
    elif ((350 <= k <= 370) and (Fext2 is not None)):
        x = rk4(x, u, dt, Fext2)
    else:
        x = rk4(x, u, dt, None)

    xs.append(np.copy(x))

print("Program found the solution!")
# print("Wait for simulation...")
# simulation(xs, x0, x_ref)

In [ ]:
# --- Cell 10 ---
# Let's plot it!
xs = np.array(xs)
us = np.array(us)

fig = plt.figure(figsize=(15, 5))
# fig.suptitle("Results without $F_{ext}$ at hover point $(x,y,θ) = (16, 84.5, 0)$ and $v_{max} = 85$", fontsize=10, fontweight='bold', y=0.95)
# fig.suptitle("Results without $F_{ext}$ at hover point $(x,y,θ) = (-16, 84.5, 0)$ and $v_{max} = 85$", fontsize=10, fontweight='bold', y=0.95)
# fig.suptitle("Results without $F_{ext}$ at hover point $(x,y,θ) = (15.5, 19.5, 0)$ and $v_{max} = 20$", fontsize=10, fontweight='bold', y=0.95)
# fig.suptitle("Results without $F_{ext}$ at hover point $(x,y,θ) = (-15.5, 19.5, 0)$ and $v_{max} = 20$", fontsize=10, fontweight='bold', y=0.95)
fig.suptitle("Results with $F_{ext}$ at hover point $(x,y,θ) = (15, 4.5, 0)$ ", fontsize=10, fontweight='bold', y=0.95)

ax1 = fig.add_subplot(131)
ax1.plot([n*dt for n in range(len(xs))], xs[:,0], label=r'x')
ax1.plot([n*dt for n in range(len(xs))], xs[:,1], label=r'y')
ax1.plot([n*dt for n in range(len(xs))], xs[:,2], label=r'θ')
ax1.legend()

ax2 = fig.add_subplot(132)
ax2.plot([n*dt for n in range(len(xs))], xs[:,3], label=r'$\dot{x}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,4], label=r'$\dot{y}$')
ax2.plot([n*dt for n in range(len(xs))], xs[:,5], label=r'$\dot{θ}$')
ax2.legend()

ax3 = fig.add_subplot(133)
ax3.plot([n*dt for n in range(len(us))], us[:,0], label=r'T')
ax3.plot([n*dt for n in range(len(us))], us[:,1], label=r'τ')
ax3.legend()

# fig.savefig('file_name.png', dpi=300, bbox_inches='tight')

plt.show()

4. Evaluate how far from the linearization point, the LQR controller is able to control the system.

In [ ]:
# --- Cell 11 ---
# Εquilibrium state and control
x_bar = np.array([[0., 0., 0., 0., 0., 0.]]).T
u_bar = np.array([[m*g, 0.]]).T

# Initial state and control
x0 = x_bar
u0 = u_bar

# Reference control
u_ref = np.array([[m*g, 0.]]).T

# Linearization around (x_bar, u_bar) 
A, B = linearize(x_bar, u_bar, dt)

# LQR
K, _ = lqr(A, B, QN, Q, R)

x = np.copy(x0)
xs = [np.copy(x)]
us = []

'''
    Try to change:
    - x_target
    - max_vel
    - Fext1
    - Fext2
    to evaluate how far from the linearization point the LQR controller
    is able to control the system with specific max velocity value and external forces!
'''
# Reference value of x
x_target = 15

# List with test values of y
y_values = np.arange(0, 200.5, 0.5)

# Maximum velocity vx/vy
max_vel = 20

# External force / Disturbance
Fext1 = None
Fext2 = None
# Fext1 = np.array([[-25., -10., 0.02]]).T # [fx, fy, f_theta]
# Fext2 = np.array([[9., 10., 0.05]]).T # [fx, fy, f_theta]

# Maximum height of quadrotor that the system is still stable
max_stable_y = -1

print(f"Starting stability search...")
for y_target in y_values:
    # Reference state
    x_ref = np.array([[x_target, y_target, 0., 0., 0., 0.]]).T

    # Current state
    x = np.copy(x0)
    
    # Flag!
    unstable_point = False 
    
    for k in range(S-1):
        # Control update
        u = u_bar - K @ (x - x_ref)

        # Clamp u
        T = max(Tmin, min(u[0,0], Tmax))
        tau = max(tau_min, min(u[1,0], tau_max))
        u = np.array([[T, tau]]).T
        
        # State update (with the actual non-linear dynamics) + Apply external force/disturbance
        if ((60 <= k <= 80) and (Fext1 is not None)):
            x = rk4(x, u, dt, Fext1)
        elif ((350 <= k <= 370) and (Fext2 is not None)):
            x = rk4(x, u, dt, Fext2)
        else:
            x = rk4(x, u, dt, None)
        
        # Stability check
        if (np.max(np.abs(x)) > max_vel):
            unstable_point = True
            break
    
    if unstable_point:
        continue 
    else:
        dist = np.linalg.norm(x - x_ref)
        if (dist < 2.):
            max_stable_y = y_target
        else:
            break
print(f"The system is stable at: x = {x_target}, y = {max_stable_y}")